<a href="https://colab.research.google.com/github/Zhennue/Final_Assignment_DAM101/blob/main/3rd_week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision gradio librosa numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os

def delete_non_npy_files(root_dir):
    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            if not filename.endswith('.npy'):
                file_path = os.path.join(foldername, filename)
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")

# Replace this with your directory path
root_directory = "/content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram"
delete_non_npy_files(root_directory)


Deleted: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram/Dzongkha/SONAM WNGDI TOP SONGS 2022 ｜｜ BHUTANESE NEW SONGS 2022｜｜_chunk5_clip1_10s_01.wav
Deleted: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram/Dzongkha/PHUDOSA- Sonam Wangchen ft. Tashi Kheldon Dorji and The Monarchs @YeshiLhendupFilms ｜ Music Video_song6_clip1_10s_01.wav
Deleted: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram/Dzongkha/Latest Hit Song ｜｜ Bhutanese Popular song_song3_clip3_10s_01.wav
Deleted: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram/Dzongkha/Aum Penjor ｜ Full song  ｜ BTN Tshering Dorji ｜ Karma Jerry ｜ Tandin Bidha ｜ Ugyen Seldon ｜_part1_10s_01.wav
Deleted: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram/Dzongkha/SONAM WNGDI TOP SONGS 2022 ｜｜ BHUTANESE NEW SONGS 2022｜｜_chunk4_clip1_10s_01.wav
Deleted: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram/Dzongkha/PHUDOSA- Sonam Wangchen ft. Tashi Kheldon Dorji and 

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from glob import glob
from tqdm import tqdm  # nice progress bar

# ==== CONFIG ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_weights_path = "/content/drive/MyDrive/SS2025/DAM101/Assignment_4/best_model.pt"
val_data_path = "/content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram"

num_classes = 4
expected_channels = 4
target_shape = (4, 128, 313)
batch_size = 8

# ==== Dataset ====
class AudioFeatureDataset(Dataset):
    def __init__(self, root_dir, target_shape=target_shape):
        all_files = glob(os.path.join(root_dir, "*/*.npy"))
        self.label_map = {'English': 0, 'Dzongkha': 1, 'Hindi': 2, 'Sharchogpa': 3}
        self.target_shape = target_shape
        self.filepaths = []

        for fp in all_files:
            try:
                data = np.load(fp, allow_pickle=True)
                if data.ndim in [2, 3]:
                    self.filepaths.append(fp)
            except Exception as e:
                print(f"Skipping file {fp} due to error: {e}")

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, idx):
        filepath = self.filepaths[idx]
        data = np.load(filepath, allow_pickle=True)

        if data.ndim == 2:
            data = np.expand_dims(data, axis=0)
        elif data.ndim > 3:
            raise ValueError(f"Invalid shape: {filepath}")

        # Adjust channels
        if data.shape[0] < expected_channels:
            pad = np.zeros((expected_channels - data.shape[0], data.shape[1], data.shape[2]))
            data = np.concatenate((data, pad), axis=0)
        elif data.shape[0] > expected_channels:
            data = data[:expected_channels]

        # Crop or pad spatial dims
        padded = np.zeros(self.target_shape)
        h, w = min(data.shape[1], self.target_shape[1]), min(data.shape[2], self.target_shape[2])
        padded[:, :h, :w] = data[:, :h, :w]

        label_name = os.path.basename(os.path.dirname(filepath))
        label = self.label_map.get(label_name, -1)
        if label == -1:
            raise ValueError(f"Unknown label: {label_name} for file {filepath}")

        return torch.tensor(padded).float(), label

# ==== Model Definition ====
import torchvision.models as models

class ResNet34Custom(nn.Module):
    def __init__(self, in_channels=4, num_classes=4):
        super().__init__()
        base = models.resnet34(pretrained=False)
        base.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        base.fc = nn.Linear(base.fc.in_features, num_classes)
        self.net = base

    def forward(self, x):
        return self.net(x)

# ==== Load model weights ====
def load_model(path, in_channels, num_classes):
    model = ResNet34Custom(in_channels, num_classes)
    try:
        state_dict = torch.load(path, map_location=device)
        new_state_dict = {}
        for k, v in state_dict.items():
            if k.startswith("module."):
                new_state_dict[k[7:]] = v
            else:
                new_state_dict[k] = v
        model.load_state_dict(new_state_dict, strict=True)
    except Exception as e:
        print(f"Error loading weights: {e}")
        return None

    model.to(device)
    model.eval()
    print("Model loaded successfully!")
    return model

# ==== Evaluate ====
def evaluate(model, dataloader):
    if model is None:
        print("Model is None, aborting evaluation.")
        return 0.0

    correct = 0
    total = 0

    print(f"Evaluating on {len(dataloader.dataset)} samples with batch size {dataloader.batch_size} ...")

    for i, (x, y) in enumerate(tqdm(dataloader, desc="Evaluating")):
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)
        preds = outputs.argmax(dim=1)

        correct += (preds == y).sum().item()
        total += y.size(0)

        if (i+1) % 10 == 0 or i == 0:
            print(f"Batch {i+1} Accuracy so far: {correct/total:.4f}")

    final_acc = correct / total if total > 0 else 0
    print(f"Final Accuracy on validation set: {final_acc:.4f}")
    return final_acc

def main():
    print("Loading dataset...")
    val_dataset = AudioFeatureDataset(val_data_path)
    print(f"Loaded {len(val_dataset)} samples")

    # Sanity check first 3 samples
    for i in range(min(3, len(val_dataset))):
        x, y = val_dataset[i]
        print(f"Sample {i}: x shape = {x.shape}, label = {y}")

    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = load_model(model_weights_path, expected_channels, num_classes)
    acc = evaluate(model, val_loader)

if __name__ == "__main__":
    main()

Loading dataset...
Loaded 16851 samples
Sample 0: x shape = torch.Size([4, 128, 313]), label = 1
Sample 1: x shape = torch.Size([4, 128, 313]), label = 1
Sample 2: x shape = torch.Size([4, 128, 313]), label = 1
Model loaded successfully!
Evaluating on 16851 samples with batch size 8 ...


Evaluating:   0%|          | 1/2107 [00:01<50:32,  1.44s/it]

Batch 1 Accuracy so far: 0.0000


Evaluating:   0%|          | 10/2107 [00:10<33:27,  1.04it/s]

Batch 10 Accuracy so far: 0.0000


...

Batch 2090 Accuracy so far: 0.0789


Evaluating: 100%|█████████▉| 2100/2107 [36:22<00:08,  1.25s/it]

Batch 2100 Accuracy so far: 0.0785


Evaluating: 100%|██████████| 2107/2107 [36:27<00:00,  1.04s/it]

Final Accuracy on validation set: 0.0783


In [ ]:
import torch
import torch.nn as nn
from torchvision import models
import gradio as gr
import numpy as np
import librosa
import json
import os
from skimage.transform import resize  # for resizing features

# ========== Model Definition ==========
class ResNet34Custom(nn.Module):
    def __init__(self, in_channels=4, num_classes=4):
        super().__init__()
        base = models.resnet34(weights=None)
        base.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        base.fc = nn.Linear(base.fc.in_features, num_classes)
        self.net = base

    def forward(self, x):
        return self.net(x)

# ========== Configurations ==========
languages = ['english', 'hindi', 'dzongkha', 'sharchogpa']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ========== Load Model ==========
model = ResNet34Custom(in_channels=4, num_classes=len(languages))
model_path = '/content/drive/MyDrive/SS2025/DAM101/Assignment_4/best_model.pt'

try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
except Exception as e:
    raise RuntimeError(f"Failed to load model: {e}")

# ========== Load Best Hyperparameters ==========
tuning_results_path = '/content/drive/MyDrive/SS2025/DAM101/Assignment_4/tuning_results.json'
try:
    with open(tuning_results_path, 'r') as f:
        best_hparams = json.load(f)
except Exception as e:
    best_hparams = {"error": f"Could not load hyperparameters: {e}"}

# ========== Feature Extraction ==========
def extract_features(audio, sr=22050, n_mels=224, n_mfcc=224, hop_length=512):
    try:
        mel = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels, hop_length=hop_length)
        mel_db = librosa.power_to_db(mel, ref=np.max)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sr, hop_length=hop_length)
        contrast = librosa.feature.spectral_contrast(y=audio, sr=sr, hop_length=hop_length)

        def pad_resize(feat):
            # Pad or crop time dimension to 224
            if feat.shape[1] < 224:
                pad_width = 224 - feat.shape[1]
                feat = np.pad(feat, ((0, 0), (0, pad_width)), mode='constant')
            feat = feat[:, :224]
            # Resize frequency and time to (224, 224)
            feat_resized = resize(feat, (224, 224), mode='reflect', anti_aliasing=True)
            return feat_resized

        def normalize(x):
            x_min, x_max = x.min(), x.max()
            return (x - x_min) / (x_max - x_min + 1e-6)

        mel_db = normalize(pad_resize(mel_db))
        mfcc = normalize(pad_resize(mfcc))
        chroma = normalize(pad_resize(chroma))
        contrast = normalize(pad_resize(contrast))

        stacked = np.stack([mel_db, mfcc, chroma, contrast], axis=0).astype(np.float32)
        return stacked
    except Exception as e:
        raise ValueError(f"Feature extraction failed: {e}")

# ========== Prediction Function with Debug ==========
def predict(audio_path, temperature=1.0):
    """
    Predict the language of the audio.
    Parameters:
      - audio_path: str path to audio file
      - temperature: float, >0, used to soften or sharpen softmax probabilities
    """
    try:
        if not isinstance(audio_path, str) or not os.path.isfile(audio_path):
            return "Invalid or missing audio file."

        audio_np, sr = librosa.load(audio_path, sr=22050)

        if audio_np is None or not isinstance(audio_np, np.ndarray) or len(audio_np.shape) != 1:
            return "Invalid waveform format."

        if audio_np.size == 0:
            return "Empty audio file."

        features = extract_features(audio_np, sr)

        if features.shape != (4, 224, 224):
            return f"Feature shape mismatch: expected (4, 224, 224), got {features.shape}"

        x = torch.tensor(features).unsqueeze(0).to(device)

        with torch.no_grad():
            logits = model(x)
            logits_np = logits.cpu().numpy()[0]

            # Optional temperature scaling
            scaled_logits = logits / temperature
            probs = torch.softmax(scaled_logits, dim=1).cpu().numpy()[0]

        # Debug prints to check for Hindi bias
        print("Logits:", logits_np)
        print("Probabilities:", probs)

        return {lang: round(float(prob), 4) for lang, prob in zip(languages, probs)}

    except Exception as e:
        return f"Prediction error: {str(e)}"

# ========== Gradio Interface ==========
iface = gr.Interface(
    fn=predict,
    inputs=[gr.Audio(type="filepath"), gr.Slider(minimum=0.1, maximum=5.0, step=0.1, value=1.0, label="Softmax Temperature (Higher = Softer)")],
    outputs=gr.Label(num_top_classes=4),
    title=" Language Classification from Audio",
    description=(
        "Upload a song or recorded clip. The model predicts the language among:\n"
        "- English\n- Hindi\n- Dzongkha\n- Sharchogpa\n\n"
        f" Best Hyperparameters: {json.dumps(best_hparams, indent=2) if isinstance(best_hparams, dict) else best_hparams}\n\n"
        "Use the slider to adjust softmax temperature for debugging output probabilities."
    )
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://85c0c12f4c60e01d01.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
